# Impianti sportivi

Eliminare impianti di piste ciclabili e modificare il tipo di data_ini in intero del tipo : 2023

In [ ]:
import json

In [40]:
PATH_IMPIANTI_SPORTIVI_RAW = "../Data/Raw/Punti_di_interesse/impianti_sportivi_raw.geojson"
PATH_IMPIANTI_SPORTIVI_CLEAN = "../Data/Clean/Punti_di_interesse/impianti_sportivi_clean.geojson"

## ETL

Apertura file Raw

In [52]:
with open(PATH_IMPIANTI_SPORTIVI_RAW, "r", encoding="utf-8") as f:
    data = json.load(f)

Filtriamo solo le strutture che hanno una data inizio costruzione e manteniamo solo le colonne che ci interessano

## ------ MESSAGGIO PER LORENZO -------

**Scusa lore se sto cambiando mille volte il codice ma oggi non ragiono!**
Ecco la spiegazione:  
 E' molto meglio CREARE da 0 la lista di "features" che vogliamo tenere e poi sostituirla
 con quella pre-esistente del data, anziché rimuovere con pop da una copia perché
 sempre per lo stesso discorso: se eliminiamo degli elementi, anche se in delle copie, poi
 gli indici "i" tra una lista e l'altra possono non combaciare più:

In [58]:
l1 = [1,2,3,4]
l2 = [1,2,3,4]
for i in range(len(l1)):
    if i == 1 or i == 2:
        l2.pop(i)
print(l1[2], l1[3])
print(l2)

3 4
[1, 3]


Se noti quello che ci si "aspettava" è che i==1 e i == 2 fossero i numeri: 2 e 3 della prima lista e quindi il risultato della seconda fosse una lista che contiene solo [1,4]
Invece il risultato è [1,3] perché:
Nel ciclo di i == 1 viene rimosso effettivamente il numero 2 ma il numero 3 passa dall'indice 2 all'indice 1 e il numero 4 passa dall'indice 3 all'indice 3: -> l1[1,2,3,4] e l2[1,3,4]
Al ciclo i == 2, che per l1 è il numero 3 ma nella l2 è il numero 4!!! e quindi viene rimosso il 4 anziché il 3  
-> l2[1,3] anziché avere l2[1,4]
Spero di essermi spiegato..... per questo è più "safe" creare la lista vuota di partenza e inserire solo i valori che vogliamo:

In [ ]:
l1 = [1,2,3,4]
l2 = []
for i in range(len(l1)):
    if not (i == 1 or i == 2):  # ho messo not in modo da Inserire tutti tranne quelli (che prima abbiamo invece tolto)
        l2.append(l1[i])
print(l1[2], l1[3])
print(l2)   # Risultato corretto: 1,4

## ------ FINE SPIEGAZIONE ------

In [ ]:
new_feature = []
for feature in data["features"]:
    if feature["properties"]["data_ini"]:
        new_feature.append({
            "type": "Feature",
            "properties":{
                "municipio": feature["properties"]["municipio"],
                "località": feature["properties"]["località"],
                "id_impianto": feature["properties"]["obj_id"], 
                "data_ini": feature["properties"]["data_ini"],
                "area_mq": feature["properties"]["area_mq"],
                "perim_m": feature["properties"]["perim_m"],
                "descrizione_codice": feature["properties"]["descrizione_codice"]
                },
            "geometry": feature["geometry"]
            }
        )
data["features"] = new_feature

Salvataggio in Clean

In [56]:
with open(PATH_IMPIANTI_SPORTIVI_CLEAN, "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=2)